## 2.9. 데이터세트

**허깅페이스 데이터세트의 매게변수**

* **path**: 데이터세트의 경로 또는 이름을 설정한다. 이 매개변수의 값에 따라 load_dataset함수가 사용하는 데이터세트 빌더가 결정된다. 로컬 데이터세트의 경우, 해당 디렉토리의 경로를 지정한다. path가 로털데이터세트 스트립트파일이거나 동일한 이름의 디렉토리인 경우 스크립트에서 데이터를 불러온다.
* **name**: 데이터세트의 이름을 설정한다. 공개 데이터세트에 여러 종류가 있다면 특정 구성을 선택해 불러올 수 있다.
* **data_dir**: 데이터세트를 저장할 디렉토리를 설정한다. 이 매개변수를 사용하면 데이터세트를 다운로드할 디렉토리를 지정할 수 있다.
* **data_files**: 데이터세트의 파일 경로를 지정한다. `data_dir`는 디렉토리를 지정하고, `data_files`는 파일 이름을 지정한다. 특정 경로의 특정 파일만 불러올 수 있다.
* **split**: 데이터세트의 분할을 지정한다. `train`, `test`, `train+test` `train[80%]+test[20%]` 등을 지정할 수 있다.
* **cache_dir**: 데이터세트를 캐시할 디렉토리를 지정한다. 이 매개변수를 사용하면 데이터세트를 다운로드한 후 캐시할 디렉토리를 지정할 수 있다.
* **revision**: 데이터세트의 버전을 지정한다. 이 매개변수를 사용하면 데이터세트의 특정 버전을 지정할 수 있다.
* **token**: 데이터세트의 API 토큰을 지정한다. 이 매개변수를 사용하면 데이터세트를 다운로드할 때 사용할 API 토큰을 지정할 수 있다.
* **streaming**: 데이터세트를 스트리밍할지 여부를 지정한다. 이 매개변수를 사용하면 데이터세트를 다운로드하지 않는다.
* **num_proc**: 로컬에서 데이터세트를 다운로드하고 생성할 때 사용할 프로세스 수를 설정한다.



In [3]:
# 데이터세트 불러오기

from datasets import load_dataset
datasets1 = load_dataset("squad")
datasets2 = load_dataset("squad", split="train[:10]+validation[:10]")

print(datasets1)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


In [4]:
print(datasets2)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 20
})


첫번째 데이터세트는 전체를 가져오고 두번째는 처음 10개의 학습데이터와 처음 10개의 검증데이터를 결합해 새로운 데이터세트를 생성한다.

**데이터세트 불러오기**
```python
from datasets import load_dataset

dataset1 = load_dataset(path="../datasets")
dataset2 = load_dataset(path="json", data_dir="../datasets")
dataset3 = load_dataset(path="json", data_files={"train": "../datasets/train.json", "test": "../datasets/test.json"})
```

`dataset1`은 전체 데이터세트를 불러오고, `dataset2`는 `json`데이터세트를 불러온다. `dataset3`은 `train.json`과 `test.json`파일을 불러온다.

### 2.9.1. 선택, 분리, 병합

전체 데이터세트를 사용하지 않고 일부분만 사용하거나, 여러 데이터세트를 결합하는 경우가 있다. 허깅페이스 데이터세트는 이러한 경우를 위한 기능을 제공한다.

**데이터세트 관리**
* **데이터세트 선택**: 인덱스나 조건을 지정해 원하는 데이터 샘플만 선택할 수 있다. 특정 범위의 데이터나 랜덤 샘플링 등에 활용할 수 있다.
* **데이터세트 분할**: 대규모 데이터세트를 여러 조각으로 분할할 수 있다. 병렬 처리나 분산 학습에 활용한다.
* **데이터세트 병합**: 여러 데이터세트를 결합해 새로운 데이터세트를 생성할 수 있다. 학습 데이터와 검증 데이터를 결합해 새로운 데이터세트를 생성할 수 있다.

In [5]:
# 데이터세트 선택, 분리 병합

from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("squad", split="train[:10]")

chunk1 = dataset.select([0,1])
chunk2 = dataset.shard(num_shards=2, index=0)
chunk3 = dataset.shard(num_shards=2, index=1)
concat_dataset= concatenate_datasets([chunk1, chunk2, chunk3])

print(len(chunk1))
print(len(chunk2))
print(len(chunk3))
print(len(concat_dataset))

2
5
5
12


### 2.9.2. 필터 및 맵

**filter와 map의 장점(?)**

* **효율성**: 병렬 처리를 지원해 대규모 데이터세트에 대한 전처리 작업 속도를 크게 향상시킨다. 배치 옵션을 사용하면 여러 데이터를 병렬로 처리할 수 있다.
* **간결성**: 단일 함수 호출로 데이터세트의 모든 예제에 대해 원하는 전처리 작업을 수행할 수 있다.
* **유연성**: 다양한 전처리 함수를 허용하므로, 전처리 파이프라인을 구축할 수 있다. 여러 개의 전처리 함수를 연결하거나 lambda함수를 사용할 수 있다.
* **메모리 효율성**: 전처리된 데이터를 바로 반환하므로, 중간 데이터 구조를 저장할 필요가 없다.


In [ ]:
# 데이터세트 filter, map

from datasets import load_dataset
from transformers import BertTokenizer

def filter_short_context(data):
    """
    컨텍스트의 길이가 1024이하인 데이터만 필터링
    """
    return len(data["context"]) > 1024

def tokenize_function(tokenizer, data):
    """
    질문과 컨텍스트를 연결하고 토크나이저를 사용해 토큰화 및 정수 인코딩을 수행한다. [SEP]을 사용해 질문과 컨텍스트 문장을 구별한다.
    """
    inputs=[
        f"{question} [SEP] {context}" for question, context in zip(data["question"], data["context"])
    ]

    model_inputs = tokenizer(inputs,
                             max_lengt=5123,
                             padding="max_length",
                             trucation=True,
                             return_tensors="pt")
    return model_inputs

dataset = load_dataset("squad", split="train[:10]")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

  
# filter 메서드를 호출해 filter_short_context를 모든 데이터에 적용한다. 이를 통해 길이가 1,024 이하인 데이터만 유지한다.
filtered_dataset = dataset.filter(filter_short_context)

# map 메서드를 호출해 tokenize_function을 필터링된 데이터에 적용한다 batched=True 옵션을 사용하면 병렬 처리가 가능해져 속도가 빨라진다.
# 또한 remove_columns 매개변수로 원본 텍스트 열을 제거해 메모리를 절약한다.
tokenized_dataset = filtered_dataset.map(
    lambda x: tokenize_function(tokenizer, x),
    batched=True,
    remove_columns=dataset.column_names
)

# 포멧 변경 메서드를 통해 토근화된 데이터세트를 파이토치 형식으로 변환한다. 이렇게 전처리된 tokenized_dataset은 
# BertTokenizer로 토근화된 입력과 레이블을 포함하고 있다. 
tokenized_dataset.set_format(
    type="torch",
    columns=["input_ids", "token_type_ids", "attention_mask"]
)

print(tokenized_dataset)
print(tokenized_dataset[0])

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5
})
{'input_ids': tensor([  101,  2043,  2106,  1996, 24105,  2932,  1997, 10289,  8214,  4088,
         4640,  1029,   102,  2004,  2012,  2087,  2060,  5534,  1010, 10289,
         8214,  1005,  1055,  2493,  2448,  1037,  2193,  1997,  2739,  2865,
        11730,  1012,  1996,  3157,  3076,  1011,  2448, 11730,  2421,  2093,
         6399,  1010,  2119,  1037,  2557,  1998,  2547,  2276,  1010,  1998,
         2195,  7298,  1998,  9263,  1012,  5625,  2004,  1037,  2028,  1011,
         3931,  3485,  1999,  2244,  7326,  1010,  1996, 24105,  2932,  2003,
         3843,  3807,  7058,  1998,  4447,  2000,  2022,  1996,  4587,  7142,
         9234,  4772,  1999,  1996,  2142,  2163,  1012,  1996,  2060,  2932,
         1010,  1996, 26536, 17420,  1010,  2003,  2207,  3807,  1037,  2095,
         1998,  7679,  2006,  3076,  3906,  1998,  8266,  1012,  1996,  8514,
        24803,  2003,  2405,  6604

### 2.9.3. 기타 메서드

**열조작 메서드**

---
* **add_column**: 데이터세트에 새로운 열을 추가한다. 파생 특징이나 부가 정보를 열 형태로 포함시킬 수 있다.
* **rename_column**: 데이터세트의 열 이름을 변경한다. 열 이름을 변경해 데이터를 더 쉽게 이해하고 사용할 수 있다.
* **remove_column**: 데이터세트의 열을 삭제한다. 불필요한 열을 제거해 데이터를 더 간결하게 관리할 수 있다.

**데이터 정렬 및 섞기 메서드**

---
* **shuffle**: 데이터세트를 섞는다. 데이터를 무작위로 섞어 학습 데이터와 검증 데이터를 나눌 때 사용한다.
* **sort**: 데이터세트를 정렬한다. 데이터를 정렬해 특정 순서로 데이터를 사용할 수 있다.

**데이터 형식 변경 메서드**

---
* **class_encode_column**: 데이터세트의 범주형 열을 숫자로 인코딩한다. 범주형 열을 숫자로 변환해 모델 학습에 사용할 수 있다.
* **set_format**: 데이터세트의 형식을 변경한다. 데이터세트의 형식을 변경해 모델 학습에 적합한 형태로 변환할 수 있다.

**데이터세트 정보 및 특성 확인 메서드**

---
* **info**: 데이터세트의 정보를 확인한다. 데이터세트의 정보를 확인해 데이터의 구조와 특성을 파악할 수 있다.
* **unique**: 데이터세트의 고유값을 확인한다. 데이터세트의 고유값을 확인해 데이터의 특성을 파악할 수 있다.